In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 현재 디렉터리 확인
!pwd

/content


In [3]:
# import해서 불러올 폴더 디렉터리로 변경 -> 즉, 폴더의 위치
# 반드시, 파일들을 같은 폴더에 해야해요!

# %cd '폴더 저장 경로'

%cd /content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/DeepCoNN

/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/DeepCoNN


In [4]:
# 변경한 디렉터리의 파일 확인

!ls

 config.ipynb	      import_Upload_ver02_DeepCoNN.ipynb   model.ipynb	 utils.ipynb
 config.py	      import_upload_ver02_deepconn.py	   model.py	 utils.py
'import code.ipynb'   main.ipynb			   __pycache__


In [5]:
# 변경한 디렉토리로 잘 되었는지 확인
!pwd

/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/DeepCoNN


In [6]:
!pip install torch torchvision


# main

In [7]:
import os
import time

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader

from config import Config
from model import DeepCoNN
from utils import load_embedding, DeepCoNNDataset, predict_mse, date

In [8]:



# ================= Training =================
def train(train_dataloader, valid_dataloader, model, config, model_path):
    print(f'{date()}## Start the training!')
    train_mse = predict_mse(model, train_dataloader, config.device)
    valid_mse = predict_mse(model, valid_dataloader, config.device)
    print(f'{date()}#### Initial train mse {train_mse:.6f}, validation mse {valid_mse:.6f}')
    start_time = time.perf_counter()

    # weight_decay = L2 정규화
    opt = torch.optim.Adam(model.parameters(), config.learning_rate, weight_decay=config.l2_regularization)

    # 학습률 스케줄
    # ExponentialLR을 사용하여 학습률을 매 에포크마다 learning_rate_decay 비율로 줄이는 방식
    lr_sch = torch.optim.lr_scheduler.ExponentialLR(opt, config.learning_rate_decay)

    best_loss = float('inf')    # 최소값을 위한 초기값 설정 -> validation에서의 최소 mse값

    for epoch in range(config.train_epochs):
        model.train()
        total_loss, total_samples = 0, 0
        for batch in train_dataloader:
            user_reviews, item_reviews, ratings = map(lambda x: x.to(config.device), batch)
            predict = model(user_reviews, item_reviews)

            # F.mse_loss를 사용해 예측값과 실제값 간의 평균 제곱 오차(MSE)를 계산
            loss = F.mse_loss(predict, ratings, reduction='sum')
            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            total_samples += len(ratings)

        lr_sch.step()
        model.eval()
        valid_mse = predict_mse(model, valid_dataloader, config.device)
        train_loss = total_loss / total_samples
        print(f"{date()}#### Epoch {epoch:3d}; train mse {train_loss:.6f}; validation mse {valid_mse:.6f}")

        if best_loss > valid_mse:
            best_loss = valid_mse
            torch.save(model, model_path)

    print(f'{date()}## End of training! Time used {time.perf_counter() - start_time:.0f} seconds.')






# ================= Testing =================
def test(dataloader, model, config):
    print(f'{date()}## Start the testing!')
    test_loss = predict_mse(model, dataloader, config.device)
    print(f"{date()}## Test end, test mse is {test_loss:.6f}")


# ================= Main =================
if __name__ == '__main__':
    config = Config()
    print(config)
    print(f'{date()}## Load embedding and data...')
    word_emb, word_dict = load_embedding(config.word2vec_file)

    train_dataset = DeepCoNNDataset(config.train_file, word_dict, config)
    valid_dataset = DeepCoNNDataset(config.valid_file, word_dict, config, retain_rui=False)
    test_dataset = DeepCoNNDataset(config.test_file, word_dict, config, retain_rui=False)
    train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size)

    model = DeepCoNN(config, word_emb).to(config.device)

    os.makedirs(os.path.dirname(config.model_file), exist_ok=True)
    train(train_dataloader, valid_dataloader, model, config, config.model_file)
    test(test_dataloader, torch.load(config.model_file), config)



2025-01-04 04:09:30## Load embedding and data...
2025-01-04 04:09:59## Start the training!
2025-01-04 04:10:00#### Initial train mse 15.128394, validation mse 12.743154
2025-01-04 04:10:05#### Epoch   0; train mse 7.115717; validation mse 6.147247
2025-01-04 04:10:17#### Epoch   1; train mse 3.274294; validation mse 5.354558
2025-01-04 04:10:18#### Epoch   2; train mse 2.776718; validation mse 4.160897
2025-01-04 04:10:19#### Epoch   3; train mse 2.536752; validation mse 3.497633
2025-01-04 04:10:20#### Epoch   4; train mse 2.428817; validation mse 3.286261
2025-01-04 04:10:21#### Epoch   5; train mse 2.242472; validation mse 3.700950
2025-01-04 04:10:21#### Epoch   6; train mse 2.165482; validation mse 3.373932
2025-01-04 04:10:22#### Epoch   7; train mse 1.985541; validation mse 3.056689
2025-01-04 04:10:23#### Epoch   8; train mse 1.939221; validation mse 3.308526
2025-01-04 04:10:23#### Epoch   9; train mse 1.916216; validation mse 2.962361
2025-01-04 04:10:24#### Epoch  10; train 

<ipython-input-8-3797641bf3f5>:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test(test_dataloader, torch.load(config.model_file), config)
